In [14]:
import pandas as pd
import numpy as np
import nltk
import re
import datetime
import math
import pickle 

from nltk.corpus import stopwords 
from collections import OrderedDict
from itertools import islice
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [15]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate=0.1):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.weights_input_hidden = np.random.uniform(-1,1,size=(hidden_nodes, input_nodes))
        self.weights_hidden_output = np.random.uniform(-1,1,size=(output_nodes, hidden_nodes))
        self.bias_hidden = np.ones((hidden_nodes,1))
        self.bias_output = np.ones((output_nodes,1))
        self.learning_rate=learning_rate
        
    def sigmoid(self,x):
        return 1/(1+ math.exp(-x))
    
    def derivate(self,x):
        return x*(1-x)
    
   
    def feedforward(self,input_v):
        sigmoid_vector = np.vectorize(self.sigmoid)
        
        input_vector = input_v.reshape((self.input_nodes,1))
    
        hidden = np.dot(self.weights_input_hidden,input_vector)
        hidden = np.add(hidden, self.bias_hidden)
        hidden = sigmoid_vector(hidden)
    
        output = np.dot(self.weights_hidden_output, hidden)
        output = np.add(output, self.bias_output)
        output = sigmoid_vector(output)
    
        return output
    
    def backpropagation(self, input_v, target_v):
        input_vector = input_v.reshape((self.input_nodes,1))
        target_vector = target_v.reshape((self.output_nodes,1))
        
        sigmoid_vector = np.vectorize(self.sigmoid)
        derivate_vector = np.vectorize(self.derivate)
    
        hidden = np.dot(self.weights_input_hidden,input_vector)
        hidden = np.add(hidden, self.bias_hidden)
        hidden = sigmoid_vector(hidden)
        
        output = np.dot(self.weights_hidden_output, hidden)
        output = np.add(output, self.bias_output)
        output = sigmoid_vector(output)

        

        output_error = np.subtract(target_vector,output)
        error = output_error.sum(0)
        
        gradient = derivate_vector(output)
        gradient = np.multiply(gradient,output_error)
        gradient = np.multiply(gradient, self.learning_rate)
        
        hidden_transpose = np.transpose(hidden)
        weights_ho_deltas = np.dot(gradient, hidden_transpose)
        
        self.weights_hidden_output = np.add(self.weights_hidden_output, weights_ho_deltas)
        self.bias_output = np.add(self.bias_output, gradient)
        
        
        transpose_weights_hidden_output = np.transpose(self.weights_hidden_output)
        hidden_error = np.dot(transpose_weights_hidden_output, output_error)
        
    
        hidden_gradient = derivate_vector(hidden)
        hidden_gradient = np.multiply(hidden_gradient, hidden_error)
        hidden_gradient = np.multiply(hidden_gradient, self.learning_rate)
        
        input_transpose = np.transpose(input_vector)
        weights_ih_deltas = np.dot(hidden_gradient, input_transpose)
        
        self.weights_input_hidden = np.add(self.weights_input_hidden, weights_ih_deltas)
        self.bias_hidden = np.add(self.bias_hidden, hidden_gradient)

        return error
    

    def train(self, train_dataframe, epochs):
        spam = 0
        ham = 0
        iteration = 0
        error_sample = 200
        errors = []

        for i in range(epochs):
            print("Epoch", i)
            for index, row in train_dataframe.iterrows():
                spam+=(row['label_tag'])  
                input_v = row.to_numpy()
                input_v = input_v[1:len(input_v)-1]
                target_v = np.array([row['label_tag']])
                error = self.backpropagation(input_v, target_v)
                
                if iteration%error_sample == 0:
                    errors.append(error)
                    print("Iteration", iteration, "error", error)
                iteration += 1
            print("\n")
            
        ham = (len(train_dataframe)*epochs)-spam
        print(f"Spam:{spam} - Ham:{ham}")
        print("Done")  

        return np.array(errors)

In [16]:
class DataUtil:
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    @staticmethod
    def normalize_data(message):
        message = re.sub(r"\$[\d]+",'price',message)
        message = re.sub(r"\%[\d]+",'percentage',message)
        message = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",'url',message)
        message = re.sub(r"www.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",'url',message)
        message = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)",'email',message)
        message = re.sub(r'[\W\d]',' ',message)
        message = re.sub(r'[\s+]',' ',message)
        message = message.strip()
        return message

    @staticmethod
    def clean_data(message):
        message = message.lower() 
        message = DataUtil.normalize_data(message)
        words = nltk.word_tokenize(message)

        result = []
        for word in words:
            if word not in DataUtil.stop_words and len(word)>2:   
                #words = DataUtil.stemmer.stem(words[i])
                word = DataUtil.lemmatizer.lemmatize(word)
                result.append(word)  
        return result

    @staticmethod
    def order_and_take(data, key, n=None):
        data = OrderedDict(sorted(data.items(), key=lambda i: i[1][key], reverse=True))
        if n!=None:
            data = dict(islice(data.items(), n))
        return data


class DocumentReader:
    def __init__(self, document):
        self.document = document
        self.words_data = {}

    def get_words(self):
        df = pd.read_csv(self.document)
        words_list = dict()

        for index, row in df.iterrows():
            words = DataUtil.clean_data(row['message'])
            for word in words: 
                words_list[word] = 0
        return words_list  

In [17]:
class Data:
    @staticmethod
    def tf(sentences):    
        words_counter = {}
        for index, sentence in enumerate(sentences):
            words = DataUtil.clean_data(sentence)
            for word in words: 
                    if word not in words_counter.keys(): 
                        words_counter[word] = {}
                        words_counter[word]['sentences'] = {}
                    if index not in words_counter[word]['sentences'].keys():
                        words_counter[word]['sentences'][index] = 1/len(words)         
                    else:
                        words_counter[word]['sentences'][index] += 1/len(words)
        return words_counter

    @staticmethod
    def tf_idf(message):
        sentences = nltk.sent_tokenize(message)
        words_count = Data.tf(sentences)
        words_data = {}

        for key, element in words_count.items():
            words_data[key] = [0 for i in range(len(sentences))]
            idf = math.log(len(sentences)/len(element['sentences']))
            for index, sentence_ratio in element['sentences'].items():    
                words_data[key][index] = sentence_ratio * idf
        return words_data
          
    @staticmethod
    def get_inputs_count(message, words_dict):  
        words = DataUtil.clean_data(message)  
        inputs = np.zeros(len(words_dict))

        for index, key in enumerate(words_dict.keys()):
            if key in words:
                inputs[index] +=1 
        return inputs
        
    @staticmethod
    def load_unique_words(dataframe):
        unique_words = {}
        for index,row in dataframe.iterrows():
            unique_words[row['word']] = 0
        return unique_words
    

In [18]:
unique_words_df = pd.read_csv('words.csv')
unique_words = Data.load_unique_words(unique_words_df)

In [19]:
df = pd.read_csv('spamham.csv')
df["label_tag"] = df["class"].map({'ham':0, 'spam':1})
df.head()

,class,message,label_tag
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,ham,U dun say so early hor... U c already then say...,0
3,ham,"Nah I don't think he goes to usf, he lives aro...",0
4,ham,Even my brother is not like to speak with me. ...,0


In [20]:
train_set = df.sample(frac=0.8)
train_set.head()

,class,message,label_tag
2304,ham,Why de. You looking good only:-)..,0
4901,spam,Today's Offer! Claim ur ?150 worth of discount...,1
3756,ham,You still around? I could use a half-8th,0
5159,spam,Someone has contacted our dating service and e...,1
2511,ham,"Sorry,in meeting I'll call later",0


In [21]:
test_set = df.drop(train_set.index)
test_set.head()

,class,message,label_tag
4,ham,Even my brother is not like to speak with me. ...,0
5,ham,As per your request 'Melle Melle (Oru Minnamin...,0
10,ham,Eh u remember how 2 spell his name... Yes i di...,0
15,ham,Aft i finish my lunch then i go str down lor. ...,0
17,ham,Just forced myself to eat a slice. I'm really ...,0


In [22]:
output = open('nn.pkl', 'rb')
nn = pickle.load(output)
output.close()

In [23]:
acuaracy = 0
confusion_matrix = [[0 for i in range(2)] for i in range(2)]

for index, row in test_set.iterrows():
    #print(row['message'])
    input_v = Data.get_inputs_count(row['message'],unique_words)
    result = round(nn.feedforward(input_v)[0,0])

    if row['label_tag'] == result:
         acuaracy+=1
    confusion_matrix[row['label_tag']][int(result)] +=1
    #print(row['class'], result,'\n')

acuaracy /= len(test_set)
presicion = confusion_matrix[1][1]/(confusion_matrix[1][1]+confusion_matrix[0][1])
recal = confusion_matrix[1][1]/(confusion_matrix[1][1]+confusion_matrix[1][0])

print(acuaracy)
print(presicion)
print(recal)
print(confusion_matrix)

0.9865350089766607
0.9699248120300752
0.9214285714285714
[[970, 4], [11, 129]]


In [24]:
input_v = Data.get_inputs_count("PRIVATE! Your 2003 Account Statement for shows 800 un-redeemed S. I. M. points. Call 08718738002 Identifier Code: 48922 Expires 21/11/04",unique_words)
round(nn.feedforward(input_v)[0,0])

1.0